In [2]:
from model import GPTConfig, GPT
import os
import pickle
from contextlib import nullcontext
import torch
import tiktoken

In [4]:
# -----------------------------------------------------------------------------
init_from = 'resume' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
out_dir = 'out-hi-100k' # ignored if init_from is not 'resume'
start = "\n" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
num_samples = 10 # number of samples to draw
max_new_tokens = 500 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster
# -----------------------------------------------------------------------------

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)


In [5]:
# model
if init_from == 'resume':
    # init from a model saved in a specific directory
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    gptconf = GPTConfig(**checkpoint['model_args'])
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)

number of parameters: 10.80M


In [6]:
model.eval()
model.to(device)
if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)


In [9]:
meta_path = os.path.join('data', checkpoint['config']['dataset'], 'meta.pkl')
load_meta = os.path.exists(meta_path)
if load_meta:
    print(f"Loading meta from {meta_path}...")
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    # TODO want to make this more general to arbitrary encoder/decoder schemes
    stoi, itos = meta['stoi'], meta['itos']
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])

Loading meta from data/madhushala/meta.pkl...


In [10]:
# run generation
start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print(decode(y[0].tolist()))
            print('---------------')



 इस कारण हाला में नहीं जा सकता है किसी की हाला,
दो बार भी हाला,
चांदी की हाला,
भाग्य को मजबूत करती है मधुशाला।।१४।
 लागत की मेरी हाला,
ही कोई आगे बढ़ाता ठीक
वर्णित है पीने वाली बात करता है जाता यह आगे है मधुशाला।।९७।
 उसने सुनती ही मेरे नाम पर मैं थिरकता और हाय, मधुशाला।।८७।
 ज्वाला मधुशाला।।१७।
 पनु व्यवहार लगभग आया यह प्याला,
जिसने इतनी दो साल है मधुओं के वजन का बाजार में सूखा पड़ा,
दो साल पहले थे प्याला,
भारत से पीकर मदिरालय पर मंगल प्याला,
उसके दो साल के बीच मेरे परिवार में है भारत से कहते थ
---------------

 दलित छोड़ प्याला,
साकीबाला, दुनिया में प्याला,
साकीबाला, क्या है अपनी जिम्मेदारी
साकीबाला, माँ को बढ़ा प्याला,
प्याला, पीनेवाले चार्टर फिल्म की हाला,
क्या बनेगा प्याला,
5 महीने पहले ही किस्मत की गई जब प्याला,
यह तो न ही प्याला,
चाहे वो छोड़ ले सकते हैं हाथों में साकीबाला,
साकीबाला, चारों दूर जब झूम लेता ज़िसको पीनेवाले,
जब तक हाला, अभी तक किस्मत के एक प्याले हाथों में हट जाएगी साकीबाला,
एक ही नहीं मैंने मंगलवार को न माँगा हूँ कि इस हाला में अभी तक यह गुणवत्ता के जरिए रह जाएगा,